In [2]:
import cv2, time

In [3]:
classNames = []
classFile = "dependencies/coco.names"
configPath = "dependencies/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
weightsPath = "dependencies/frozen_inference_graph.pb"

In [5]:
with open(classFile,"rt") as f : 
    classNames = f.read().rstrip("\n").split("\n")

model = cv2.dnn_DetectionModel(weightsPath,configPath)
model.setInputParams(size=(320, 320), scale=1.0/127.5, mean=(127.5,127.5,127.5) ,swapRB=True)

# net.setInputSize(320,320)
# net.setInputScale(1.0/ 127.5)
# net.setInputMean((127.5, 127.5, 127.5))
# net.setInputSwapRB(True)

In [35]:
def getObjects(img, thres, nms, draw=True, objects=[]) :
    classIds, confs, bbox = model.detect(img,confThreshold=thres,nmsThreshold=nms)
    #print(classIds,bbox)
    if len(objects) == 0 : objects = classNames
    objectInfo =[]
    if len(classIds) != 0 : 
        for classId, confidence, box in zip(classIds.flatten(),confs.flatten(),bbox) :
            #confidence
            className = classNames[classId - 1]
            if className in objects: 
                objectInfo.append([box,className])
                if (draw):
                    cv2.rectangle(img,box,color=(0,255,0),thickness=2)
                    cv2.putText(img,classNames[classId-1].upper(),(box[0]+10,box[1]+30),
                    cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                    cv2.putText(img,str(round(confidence*100,2)),(box[0]+200,box[1]+30),
                    cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)  
    
    return img, objectInfo

In [48]:
if __name__ == "__main__":

    cap = cv2.VideoCapture("test_data/B103-3.mp4")
    cap1 = cv2.VideoCapture("test_data/B103.mp4")
    
    cap.set(3,640)
    cap.set(4,480)
  
    #cap.set(CV_CAP_PROP_BUFFERSIZE, 3);
    
    cap1.set(3,640)
    cap1.set(4,480)
    
    #cap.set(10,70)
    
    
    while True:
        success, img = cap.read()
        result, objectInfo = getObjects(img,0.45,0.2, objects=['person'])
        
#         success1, img1 = cap1.read()
#         result1, objectInfo1 = getObjects(img1,0.45,0.2, objects=['person'])
        #print(result)
        
        if success :
            cv2.imshow("Output",img)
            
#         if success1 :
#             cv2.imshow("Output1",img1)
            
        #cv2.waitKey(1) 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cap1.release()
            cv2.destroyAllWindows()
            break